<a href="https://colab.research.google.com/github/ben854719/Cybersecurity-Risk-Agentic-AI/blob/main/Cybersecurity_Risk_to_Agentic_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langgraph google-generativeai

In [ ]:
import google.generativeai as genai

# Configure Gemini API
genai.configure(api_key="GeminiAPI_key")

def security_analysis(input_text):
    response = genai.GenerativeModel("gemini-pro").generate_content(input_text)
    return response.text


In [ ]:
import langgraph
from langgraph.graph import StateGraph

# Define the state
class SecurityState:
    text: str

# Create the graph
workflow = StateGraph(SecurityState)

# Add a security analysis node
workflow.add_node("security_step", security_analysis)

# Define execution flow
workflow.set_entry_point("security_step")

# Compile the graph
graph = workflow.compile()

In [41]:
!pip install langgraph google-generativeai langchain unstructured[local-inference] --upgrade

In [66]:
!pip install --upgrade langgraph

In [67]:
!pip install --upgrade langgraph google-generativeai unstructured

In [73]:
!pip install langgraph google-generativeai unstructured[local-inference] textblob

In [90]:
import os

os.environ["GeminiAPI_key"] = "ben_gemini"

In [104]:
import google.generativeai as genai

api_key = "ben_gemini"
genai.configure(api_key=api_key)

In [5]:
import requests
from unstructured.partition.html import partition_html
from langgraph.graph import StateGraph
import google.generativeai as genai
import os

# Secure API Key Handling
api_key = os.getenv("ben_gemini")
if not api_key:
    raise ValueError("API key not found. Set GEMINI_API_KEY as an environment variable.")
genai.configure(api_key=api_key)

# Function to fetch and process web content
def fetch_and_process_web_content(state):
    try:
        url = state.get("url")
        if not url:
            raise ValueError("URL missing from state.")

        response = requests.get(url, timeout=20)
        response.raise_for_status()

        # Extract raw text from HTML
        elements = partition_html(text=response.text)
        text_content = "\n".join([str(el) for el in elements])

        return {"url": url, "text": text_content}

    except (requests.exceptions.RequestException, ValueError) as e:
        return {"url": state.get("url", ""), "text": f"Error: {e}"}
    except Exception as e:
        return {"url": state.get("url", ""), "text": f"Unexpected error: {e}"}

# Function to summarize content with Gemini AI
def summarize_with_gemini(state):
    try:
        text_content = state.get("text", "")
        if not text_content:
            return {"url": state["url"], "text": "No text found for summarization."}

        ai_response = genai.GenerativeModel("gemini-1.5-pro").generate_content(
            f"Summarize the main points of this web page: {text_content}"
        )

        return {"url": state["url"], "summary": ai_response.text}

    except Exception as e:
        return {"url": state["url"], "summary": f"Error summarizing content: {e}"}

# Define workflow with LangGraph
workflow = StateGraph(dict)
workflow.add_node("fetch_web_content", fetch_and_process_web_content)
workflow.add_node("summarize_content", summarize_with_gemini)
workflow.set_entry_point("fetch_web_content")
workflow.add_edge("fetch_web_content", "summarize_content")

# Compile and execute workflow
executable_graph = workflow.compile()
initial_state = {"url": "https://www.helpnetsecurity.com/2025/04/24/agentic-ai-onboarding/"}
final_state = executable_graph.invoke(initial_state)

# Print summary or error message
print(final_state["summary"])

This article argues that AI agents are becoming crucial for cybersecurity, acting as "junior analysts" to handle the overwhelming volume of security alerts.  While fast and capable of multitasking, these AI agents require training and guidance, much like human hires.  Key points include:

* **AI augments, not replaces, human analysts:** AI handles repetitive tasks like alert triage, freeing human analysts for more critical threats.
* **AI needs training and context:**  Just like junior analysts, AI agents need onboarding, mentorship, and feedback to understand an organization's specific risks, priorities, and workflows.  Without this, AI can misinterpret data and miss critical threats.
* **Adapting AI is an ongoing process:** Continuous refinement through human oversight, review of AI-generated reports, and correction of mistakes is essential.  This iterative feedback loop improves AI accuracy and reduces false positives/negatives.
* **The future of security is human-AI collaboration:*